# OLGA .tpl Heatmaps

### Импорт необходимых библиотек

In [1]:
import os
import numpy as np
import pandas as pd
import pyfas as fa
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import re
matplotlib.use('agg')

### Исходные данные + вспомогательные функции

In [2]:
timeValue = 1500                                                      #РАССМАТРИВАЕМЫЙ МОМЕНТ ВРЕМЕНИ В СЕКУНДАХ
inputPath = r'C:\Users\makso\OLGA\Heatmap3'                           #ПУТЬ К ПАПКЕ СО ВСЕМИ PARAMETRIC STUDIES
outputPath = r'C:\Users\makso\OneDrive\Рабочий стол\VISUALISATION'    #ПУТЬ К ПАПКЕ ДЛЯ ВЫГРУЗКИ ТЕПЛОВЫХ КАРТ
#_______________________________________________________________________________________________________________________________

measureUnitDict = {'GOR': 'sm3/sm3', 'Pbh': 'bara', 'Pres':'bara',    # ЕСЛИ НУЖНОГО ПАРАМЕТРА НЕТ В СЛОВАРЕ,
                   'Pwh':'bara','PI':'sm3/d/bar','QGST':'sMm3/d',     # ДОБАВИТЬ ЕГО С ЕД. ИЗМ. ПО АНАЛОГИИ С ДРУГИМИ
                   'QLST':'sm3/d','QOST':'sm3/d','Tbh':'degC',
                   'Tres':'degC','Twh':'degC','WC':'%'}

fileList = []
for file in os.walk(inputPath):                                       # ПОЛУЧЕНИЕ СПИСКА СО ВСЕМИ ПАПКАМИ И ФАЙЛАМИ
        fileList.append(file)                                         # ВНУТРИ InputPath. 
                                                                      
for i in range(1,len(fileList)):                                      # УДАЛЕНИЕ В ЛИСТЕ ФАЙЛОВ С РАСШИРЕНИЕМ НЕ .tpl 
    for file in fileList[i][2]:
        if file.endswith('ppl') == True: 
            fileList[i][2].remove(file)
        
for i in range(1,len(fileList)):
    for file in fileList[i][2]:
        if file.endswith('rsw') == True: 
            fileList[i][2].remove(file)
            
for i in range(1,len(fileList)):
    for file in fileList[i][2]:
        if file.endswith('out') == True: 
            fileList[i][2].remove(file)
            
def find_nearest(array, value):                                       # ФУНКЦИЯ ДЛЯ НАХОЖДЕНИЯ ВРЕМЕНИ, БЛИЖАЙШЕГО К 
    array = np.asarray(array)                                         # ПРИСВОЕННОМУ timeValue
    idx = (np.abs(array - value)).argmin()
    return array[idx]

### Построение тепловых карт + экспорт

In [27]:
for i in range(1,len(fileList)):                                      # ИТЕРАЦИЯ ПО ПАПКАМ
    df = fa.Tpl(str(fileList[i][0]) + "/" + fileList[i][2][0])
    df = df.view_trends()
    
    positionList = df['Position'].tolist()
    positionListMod = []
    for file in positionList: positionListMod.append(re.split('_', file)[-1])
        
    indexList = df['Index'].tolist()
    parameterList = df['Variable'].tolist()
    unitList = df['Unit'].tolist()
    constantList = re.findall(r'\((.*?)\)', fileList[i][2][0])               
    n=re.findall(r'\d\.\d+|\d+',constantList[0], re.I)                       
    a=re.findall(r'[a-zA-Z]+',constantList[0])                               
    
    constantString = ''
    for j in range(0,len(a)): constantString+=a[j]+'='+n[j]+' '+str(measureUnitDict.get(a[j]))+'  '
    
    for q in range(len(parameterList)):                               # ИТЕРАЦИЯ ПО ПАРАМЕТРАМ
        listOfTpl = []
        xValueList = []
        yValueList = []
        y = 0
        mapSize = int(len(fileList[i][2])**0.5)
        x = np.empty((mapSize,mapSize))
    
        for file in fileList[i][2]:                                   # ИМПОРТ НЕОБХОДИМЫХ ДАННЫХ ИЗ ФАЙЛОВ В fileList
            string = str(file).replace('.tpl','')
            reList = re.split('_',string)
            xParameter = reList[2]
            yParameter = reList[-2]
            xUnit = measureUnitDict.get(xParameter)
            yUnit = measureUnitDict.get(yParameter)
            xValueList.append(reList[-3])
            yValueList.append(reList[-1])
        
        for z in range(mapSize):                                      # ЗАПОЛНЕНИЕ МАТРИЦЫ ДАННЫХ X
            for j in range(mapSize):
                listOfTpl.append(fa.Tpl(str(fileList[i][0]) + "/" + fileList[i][2][y]))
                f = listOfTpl[y]
                parameter = parameterList[q]
                parameterInd = indexList[q]
                parameterUnit = unitList[q]
                f.extract(parameterInd)
                x[j][z]=f.data[parameterInd][np.where(f.time ==find_nearest(f.time, timeValue))]
                y+=1
                
        sns.set(rc={'figure.figsize':(6.5,7.5)})                      # ВИЗУАЛИЗАЦИЯ ДАННЫХ МАТРИЦЫ X
        ax = sns.heatmap(x, cmap = 'Blues', linewidths = 0.7, linecolor = 'white', 
                            square = True, annot = True, fmt = '.3f', cbar = False, robust = True)
        ax.xaxis.tick_bottom()
        ax.yaxis.tick_left()
        xticks_labels = [xValueList[mapSize*n] for n in range(0, mapSize)]
        yticks_labels = [yValueList[n] for n in range(0, mapSize)]
        plt.xticks(np.arange(mapSize) + .5, labels=xticks_labels)
        plt.yticks(np.arange(mapSize) + .5, labels=yticks_labels, rotation = 0)
        plt.xlabel(str(xParameter) + ', ' + str(xUnit))
        plt.ylabel(str(yParameter) + ', ' + str(yUnit))
        plt.suptitle('\n{} [{}]\n \nPosition ={}        Time = {} sec.'.format(parameter, parameterUnit,positionListMod[q], 
                                                                               timeValue), fontsize = 14)
        plt.title(constantString, fontsize = 10)
        fig = ax.get_figure()                                        # СОХРАНЕНИЕ КАРТЫ В outputPath
        fig.savefig(outputPath + "/Study[{}]_{}_{}({}, {})".format(i, parameter,
                                             positionListMod[q],xParameter, yParameter), dpi=200)
        plt.close()
        plt.clf()